In [69]:
#-*- encoding: utf-8 -*-

import os
import numpy as np
import imp
import sys

import utils
imp.reload(utils)
from utils.utils import rescale, softmax

import dataset
imp.reload(dataset)
from dataset import MNIST, Feature

import model
imp.reload(model)   # 不这样reload，调试的时候修改引用的py文件是没有作用的
from model import Net_1, Net_2, Optim, AccFunc, Population, CrossEntropyLoss


In [58]:
# network parameters
input_size = 100
layer1_node = 20
layer2_node = 50
output_size = 2

batch_size = 1000
epoch = 1000

individual_num = 20
mu_p = 0.8

In [3]:
############### data pre-processing ###############

train_set = MNIST('mnist', 'train', (10, 10))
test_set = MNIST('mnist', 't10k', (10, 10))

'''10 class
train_feature = Feature(train_set.data, kernel_size=(4,4), stride=(3,3))
train_fv = train_feature._data['images'].reshape(-1, 100)         # 10 class, 100 input
rescale(train_fv, 30, 250, False)
train_label = train_feature._data['labels']
input_train_data = train_feature.cut_into_batch(batch_size=1000, vector=train_fv, labels=train_label)

test_feature = Feature(test_set.data, kernel_size=(4,4), stride=(3,3))
test_fv = test_feature._data['images'].reshape(-1, 100)           # 10 class, 100 input
rescale(test_fv, 30, 250, False)
test_label = test_feature._data['labels']
input_test_data = test_feature.cut_into_batch(batch_size=1000, vector=test_fv, labels=test_label)
'''

train_feature = Feature(train_set.data, kernel_size=(4,4), stride=(3,3))
train_fv, train_label = train_feature.extract_num_class(0, 1)
# print(train_fv.shape)
# print(train_label.shape)
train_fv = train_fv.reshape(-1, 100)
rescale(train_fv, 30, 250, False)
input_train_data = train_feature.cut_into_batch(batch_size=1000, vector=train_fv, labels=train_label, num_class=output_size, one_hot=True)

test_feature = Feature(test_set.data, kernel_size=(4,4), stride=(3,3))
test_fv, test_label = test_feature.extract_num_class(0, 1)
test_fv = test_fv.reshape(-1, 100)
rescale(test_fv, 30, 250, False)
input_test_data = test_feature.cut_into_batch(batch_size=1000, vector=test_fv, labels=test_label, num_class=output_size, one_hot=True)


In [4]:
input_test_data[0][1].shape
print(train_fv[:3])
print(test_fv[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  60 114  10
    0   0   0   0   0   2  76 238 210 112   0   0   0   0   0  70 230 142
   93 176   6   0   0   0  32 187  62   0   0 207  40   0   0   0 136  94
    0   0   9 198  27   0   0   0 149  43   0  28 142  72   0   0   0   0
  150 174 158 177  61   0   0   0   0   0  35 135 101  11   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  21  32
    0   0   0   0   0   0   0   3 176 134   0   0   0   0   0   0   0  93
  197  14   0   0   0   0   0   0  17 215  65   0   0   0   0   0   0   4
  172 141   1   0   0   0   0   0   0  72 226  31   0   0   0   0   0   0
    4 176 143   0   0   0   0   0   0   0   6 184  58   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  90  56   0   0
    0   0   0   0   0   1 156 115   0   0   0   0   0   0   0   1 153 166
    4   0   0   0   0   0 

In [5]:
import matplotlib.pyplot as plt

def draw(title):

    threshold = net.get_parameters()
    layer1_th = threshold[0]
    layer2_th = threshold[1]

    # print(layer1_th.shape)
    # print(layer2_th.shape)

    all_th = np.hstack((layer1_th.flatten(), layer2_th.flatten())).tolist()
    plt.hist(all_th)
    plt.title(title)
    plt.show()

In [70]:
############### model define ###############

net = Net_1(input_size, layer1_node, output_size)
# net = Net_2(input_size, layer1_node, layer2_node, output_size)

individuals = Population(individual_num, input_size, layer1_node, output_size)

acc_func = AccFunc()
criterion = CrossEntropyLoss()
optimizer = Optim(net)

In [71]:
############### train ###############

max_acc = 0.
for _ in range(epoch):
    print("Epoch: %d" % _)
    for i, (images, labels) in enumerate(input_train_data):
        individual_loss = []
        acc_collecter = []
        for param in individuals.get_all_params():
            # change different individual
            net.set_parameters(param)
            
            outputs = net(images)
            outputs = softmax(outputs)
            # print(outputs[:3])
            # print()
            # print(outputs.shape)    # shape=(batch_size, 1)

            loss = criterion(outputs, labels)
            # print(loss)
            individual_loss.append(loss)   # label should one-hot
            
            acc = acc_func(outputs, labels)
            acc_collecter.append(acc)

        ############ Top 5 acc and record #############
        acc_collecter = np.array(acc_collecter)
        acc_sorted = abs(np.sort(-acc_collecter))
        print(acc_sorted[:5])
        if acc_sorted[0] > max_acc:
            individuals.record_best(np.argsort(-acc_collecter)[0])
            max_acc = acc_sorted[0]

        ############ GA #############
        individuals.select(individual_loss) # selection good individuals
        # get next generation
        popu_params = individuals.get_all_params()
        new_popu_params = []
        for i in range(0, len(popu_params), 2):
            child_1, child_2 = individuals.crossover(popu_params[i], popu_params[i+1])
            new_popu_params.append(child_1)
            new_popu_params.append(child_2)
        
        # mutation
        popu_p = np.random.rand(individual_num)
        for i in range(individual_num):
            if popu_p[i] > mu_p:
                # print('Epoch: %d, individual: %d' % (_, i))
                individuals.mutation(new_popu_params[i])

        # update population
        individuals.update_popu(new_popu_params)

############ Save best parameters #############
individuals.save_best(max_acc, 'log')

Epoch: 0
[0.624 0.596 0.59  0.587 0.574]
[0.679 0.657 0.643 0.575 0.572]
[0.641 0.638 0.597 0.576 0.573]
[0.58  0.573 0.567 0.559 0.546]
[0.587 0.584 0.565 0.558 0.548]
[0.629 0.622 0.616 0.579 0.546]
[0.59  0.59  0.588 0.568 0.562]
[0.612 0.572 0.566 0.561 0.55 ]
[0.602 0.587 0.571 0.567 0.562]
[0.636 0.614 0.573 0.561 0.556]
[0.668 0.603 0.579 0.569 0.562]
[0.633 0.61  0.571 0.562 0.545]
[0.63609023 0.62105263 0.60902256 0.60451128 0.5443609 ]
Epoch: 1
[0.632 0.608 0.606 0.596 0.593]
[0.631 0.588 0.581 0.575 0.556]
[0.667 0.653 0.621 0.588 0.554]
[0.612 0.6   0.592 0.579 0.566]
[0.607 0.579 0.563 0.556 0.555]
[0.616 0.603 0.603 0.565 0.555]
[0.603 0.592 0.586 0.582 0.577]
[0.639 0.578 0.559 0.552 0.545]
[0.598 0.598 0.572 0.566 0.565]
[0.634 0.601 0.574 0.565 0.549]
[0.608 0.548 0.536 0.533 0.525]
[0.658 0.608 0.586 0.573 0.555]
[0.66466165 0.65864662 0.64360902 0.58195489 0.57894737]
save completed
